In [ ]:
!pip install --upgrade transformers datasets


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/week7/customerFeedback.csv'
df = pd.read_csv(file_path)

In [ ]:
print(df.head())

In [ ]:
df = df[["clean_text", "label_num"]].dropna()


In [ ]:
print(df.columns)


In [ ]:
df.rename(columns={"clean_text": "text", "label_num": "label"}, inplace=True)

In [ ]:
dataset = Dataset.from_pandas(df)


In [ ]:
dataset = dataset.train_test_split(test_size=0.2)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch['text'],
        padding=True,
        truncation=True,
        max_length=128,
        return_attention_mask=True
    )
dataset = dataset.map(tokenize, batched=True)

In [ ]:
num_labels = len(df["label"].unique())
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",          
    logging_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    report_to=[],
)


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)

    return {
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions, average='weighted'),
        'recall': recall_score(labels, predictions, average='weighted'),
        'f1': f1_score(labels, predictions, average='weighted')
    }
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

In [ ]:
trainer.evaluate()
# Label mapping according to your ratings:
label_map = {
    0: "Negative",
    1: "Satisfactory",
    2: "Good",
    3: "Excellent"
}

def predict_sentiment(text):
    # Ensure model and input are on the same device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize and move input to same device
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    ).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        predicted_class_id = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted_class_id].item()

    return label_map[predicted_class_id], confidence


In [ ]:
import matplotlib.pyplot as plt

predictions = trainer.predict(dataset["test"])

# Get the predicted labels and true labels
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(axis=1)

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=df['label'].unique())
disp.plot()
plt.title('Confusion Matrix')
plt.show()

# Additionally, we can print out the computed metrics from the evaluation step
print("Evaluation Metrics:")
print(trainer.evaluate())

In [ ]:
user_text = input("Enter a customer review: ")
label, conf = predict_sentiment(user_text)
print(f"Predicted Sentiment: {label} (Confidence(Probability): {conf:.2f})")
